In [1]:
#Importar paquetesgroup
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
#Importamos variables temporales
now = datetime.now()
ayer = datetime.now() - timedelta(days=1)
anteayer = datetime.now() - timedelta(days=2)

# la fecha de corte es la más actual
fecha_corte = now.strftime("%d") + now.strftime("%m") + now.strftime("%Y") # es la fecha actual en formato dd/mm/yyyy
print(now.strftime("%d") + '-' + now.strftime("%m") + '-' + now.strftime("%Y"))

15-04-2024


In [3]:
# Para limpiar variables duplicadas luego de un merge
def postmerge(df):
    replace_list = [i for i in df.columns if (i.endswith('_x'))]
    replace_list = [i[:-2] for i in replace_list]
    for i in replace_list:
        df[i] = df[i+'_x'].fillna(df[i+'_y'])
    df.drop([i for i in df.columns if (i.endswith('_x'))|(i.endswith('_y'))], inplace=True, axis=1)
    return df

# Para crear el COD_ID de la ejecutora
def crear_id(df, NIVEL_GOB, SECTOR, PLIEGO, EJECUTORA):
    df = df.assign(COD_ID = np.where(df[NIVEL_GOB].str.startswith('2'), (df[PLIEGO].str.split('. ', n=1).str[0] +"-" + df[EJECUTORA].str.split('. ', n=1).str[0]),
                                     (df[SECTOR].str.split('. ', n=1).str[0] +"-" + df[PLIEGO].str.split('. ', n=1).str[0] +"-"+ df[EJECUTORA].str.split('. ', n=1).str[0])))
    return df

In [4]:
bd_canon= pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Outputs', 'Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_canon_Modificada_15042024' + '.xlsx'), sheet_name='RD GG2.3')
bd_foncor = pd.read_excel(os.path.join(r'Y:\PMG 2025-2027\Outputs', 'Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_foncor_Modificada_15042024' + '.xlsx'), sheet_name='RD GG2.3')
bd_seg24 = pd.read_excel(os.path.join(r'Z:\Seguimiento del Gasto\Informacion Quincenal\2024', '2.PIAPIMDevGirxMetaEsp_2024_GR_02012024' + '.xlsx'))

C:\Users\jchamba\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
bd_canon.info()
bd_canon.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NIVEL_GOBIERNO      45 non-null     object 
 1   SECTOR              45 non-null     object 
 2   PLIEGO              45 non-null     object 
 3   FUENTE              45 non-null     object 
 4   RUBRO               45 non-null     object 
 5   TIPO_GEN_ING        45 non-null     object 
 6   CATEGORIA_GTO       45 non-null     object 
 7   GENERICA            45 non-null     object 
 8   PIA_2023            21 non-null     float64
 9   PIM_2023            21 non-null     float64
 10  DEV_2023            21 non-null     float64
 11  PIA_2024            21 non-null     float64
 12  PIM_2024            21 non-null     float64
 13  DEV_2024            21 non-null     float64
 14  PMG_2025            45 non-null     int64  
 15  PMG_2026            45 non-null     int64  
 16  PMG_2027  

,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PIA_2023,PIM_2023,...,IND_GEN_PLIEGO25,ALERTA_SUP20%_25,TOTAL_GEN_PLIEGO26,TOTAL_PLIEGO26,IND_GEN_PLIEGO26,ALERTA_SUP20%_26,TOTAL_GEN_PLIEGO27,TOTAL_PLIEGO27,IND_GEN_PLIEGO27,ALERTA_SUP20%_27
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,25436520.0,70815105.0,...,7.092679,NO,21998440,312911400,7.030246,NO,21426080,306773231,6.984338,NO
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,6134251.0,9907527.0,...,3.497591,NO,2499086,68931195,3.625479,NO,1585750,66164452,2.396680,NO
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,14027259.0,20496990.0,...,4.902645,NO,2862567,55527477,5.155226,NO,2951938,52788852,5.591972,NO
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,12219783.0,30561576.0,...,2.179599,NO,14124251,648674142,2.177403,NO,13796433,631983946,2.183035,NO
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,4577093.0,12864836.0,...,8.620262,NO,3812279,40898869,9.321233,NO,3112634,36322309,8.569483,NO


In [8]:
bd_copia1 = pd.concat([bd_canon, bd_foncor], ignore_index=True)
bd_copia1 = bd_copia1[['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGO', 'FUENTE', 'RUBRO',	'TIPO_GEN_ING',	'CATEGORIA_GTO', 'GENERICA', 'PMG_2025', 'PMG_2026', 'PMG_2027']]
bd_copia1.info()
bd_copia1.head(66)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  65 non-null     object
 1   SECTOR          65 non-null     object
 2   PLIEGO          65 non-null     object
 3   FUENTE          65 non-null     object
 4   RUBRO           65 non-null     object
 5   TIPO_GEN_ING    65 non-null     object
 6   CATEGORIA_GTO   65 non-null     object
 7   GENERICA        65 non-null     object
 8   PMG_2025        65 non-null     int64 
 9   PMG_2026        65 non-null     int64 
 10  PMG_2027        65 non-null     int64 
dtypes: int64(3), object(8)
memory usage: 5.7+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGO,FUENTE,RUBRO,TIPO_GEN_ING,CATEGORIA_GTO,GENERICA,PMG_2025,PMG_2026,PMG_2027
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,20562107,19745927,19173567
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442. GOBIERNO REGIONAL DEL DEPARTAMENTO DE APU...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,1903691,1744669,680745
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,444. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AYA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,538731,515962,451815
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,446. GOBIERNO REGIONAL DEL DEPARTAMENTO DE CUSCO,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,12488334,12265491,11918927
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,447. GOBIERNO REGIONAL DEL DEPARTAMENTO DE HUA...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,3440026,3799305,3099660
...,...,...,...,...,...,...,...,...,...,...,...
60,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,458. GOBIERNO REGIONAL DEL DEPARTAMENTO DE PUNO,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,356359,366819,384308
61,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,459. GOBIERNO REGIONAL DEL DEPARTAMENTO DE SAN...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,4032921,4269452,4506656
62,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,461. GOBIERNO REGIONAL DEL DEPARTAMENTO DE TUMBES,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,14580615,14906378,15606354
63,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,462. GOBIERNO REGIONAL DEL DEPARTAMENTO DE UCA...,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,2. OTRAS GENÉRICAS,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,8484996,8680376,9100232


In [7]:
bd_seg24.info()
bd_seg24.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244336 entries, 0 to 244335
Data columns (total 62 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ANO_EJE             244336 non-null  int64  
 1   NIVEL_GOBIERNO      244336 non-null  object 
 2   SECTOR              244336 non-null  object 
 3   PLIEGO              244336 non-null  object 
 4   EJECUTORA           244336 non-null  object 
 5   SEC_EJEC            244336 non-null  int64  
 6   DES_UE_GRUPO        241972 non-null  object 
 7   PROGRAMA_PPTAL      244336 non-null  object 
 8   TIPO_PROD_PROY      244336 non-null  object 
 9   PRODUCTO_PROYECTO   244336 non-null  object 
 10  ACT_OBRA_ACCINV     244336 non-null  object 
 11  FUNCION             244336 non-null  object 
 12  DIVISION_FUNCIONAL  244336 non-null  object 
 13  GRUPO_FUNCIONAL     244336 non-null  object 
 14  SEC_FUNC            244336 non-null  int64  
 15  FINALIDAD           244336 non-nul

,ANO_EJE,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,SEC_EJEC,DES_UE_GRUPO,PROGRAMA_PPTAL,TIPO_PROD_PROY,PRODUCTO_PROYECTO,...,GIR04,GIR05,GIR06,GIR07,GIR08,GIR09,GIR10,GIR11,GIR12,TOTAL_GIRADO
0,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,SEDE CENTRAL,0057. CONSERVACION DE LA DIVERSIDAD BIOLOGICA ...,2. PROYECTO,2474717. RECUPERACION DEL BOSQUE BASIMONTANO D...,...,0,0,0,0,0,0,0,0,0,0
1,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,SEDE CENTRAL,0057. CONSERVACION DE LA DIVERSIDAD BIOLOGICA ...,2. PROYECTO,2496438. MEJORAMIENTO DEL SERVICIO DE APOYO AL...,...,0,0,0,0,0,0,0,0,0,0
2,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,SEDE CENTRAL,0057. CONSERVACION DE LA DIVERSIDAD BIOLOGICA ...,3. PRODUCTO,3000475. AREAS NATURALES PROTEGIDAS CON CONTRO...,...,0,0,0,0,0,0,0,0,0,0
3,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,SEDE CENTRAL,0057. CONSERVACION DE LA DIVERSIDAD BIOLOGICA ...,3. PRODUCTO,3000475. AREAS NATURALES PROTEGIDAS CON CONTRO...,...,0,0,0,0,0,0,0,0,0,0
4,2024,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,001. SEDE AMAZONAS,721,SEDE CENTRAL,0057. CONSERVACION DE LA DIVERSIDAD BIOLOGICA ...,3. PRODUCTO,3000475. AREAS NATURALES PROTEGIDAS CON CONTRO...,...,0,0,0,0,0,0,0,0,0,0


In [9]:
### FORMATO TECHO ESPECIAL
Seg_2024_formato = bd_seg24.copy()
columnas = list(Seg_2024_formato.columns)
columnas = columnas[1:5] + ["FUENTE" , "RUBRO", "CATEGORIA","PROGRAMA_PPTAL"] + columnas[9:14] + ["FINALIDAD"] + columnas[24:30]
Seg_2024_formato = Seg_2024_formato[columnas + ["PIA"]]
del Seg_2024_formato['CLASE_GTO']
Seg_2024_formato = Seg_2024_formato.groupby(by = list(Seg_2024_formato.columns[:-1])).sum().reset_index()

Seg_2024_formato["PLIEGOMERGE"] =Seg_2024_formato["PLIEGO"].str[:3] 
Seg_2024_formato = Seg_2024_formato[(Seg_2024_formato["PLIEGOMERGE"].str[0]=="4")&(Seg_2024_formato["PLIEGO"].str[3]==".")]
Seg_2024_formato = Seg_2024_formato[Seg_2024_formato["FUENTE"].str[0]=="5"]
Seg_2024_formato = Seg_2024_formato[Seg_2024_formato["RUBRO"].str[:2].isin(["15","18"])]
Seg_2024_formato = Seg_2024_formato[Seg_2024_formato["GENERICA"].str[0]=="3"]
Seg_2024_formato = Seg_2024_formato[(Seg_2024_formato["SUBGENERICA_DET"].str[:1].isin(["4"]))|(Seg_2024_formato["SUBGENERICA_DET"].str[:2].isin(["11"]))|(Seg_2024_formato["SUBGENERICA_DET"].str[:4].isin(["6. R"]))]
Seg_2024_formato.rename(columns={'CATEGORIA': 'CATEGORIA_GTO'}, inplace=True)
Seg_2024_formato.info()
Seg_2024_formato.head(392)

<class 'pandas.core.frame.DataFrame'>
Index: 383 entries, 3533 to 229998
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NIVEL_GOBIERNO      383 non-null    object
 1   SECTOR              383 non-null    object
 2   PLIEGO              383 non-null    object
 3   EJECUTORA           383 non-null    object
 4   FUENTE              383 non-null    object
 5   RUBRO               383 non-null    object
 6   CATEGORIA_GTO       383 non-null    object
 7   PROGRAMA_PPTAL      383 non-null    object
 8   PRODUCTO_PROYECTO   383 non-null    object
 9   ACT_OBRA_ACCINV     383 non-null    object
 10  FUNCION             383 non-null    object
 11  DIVISION_FUNCIONAL  383 non-null    object
 12  GRUPO_FUNCIONAL     383 non-null    object
 13  FINALIDAD           383 non-null    object
 14  GENERICA            383 non-null    object
 15  SUBGENERICA         383 non-null    object
 16  SUBGENERICA_DET     383 n

,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,FUENTE,RUBRO,CATEGORIA_GTO,PROGRAMA_PPTAL,PRODUCTO_PROYECTO,ACT_OBRA_ACCINV,...,DIVISION_FUNCIONAL,GRUPO_FUNCIONAL,FINALIDAD,GENERICA,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PIA,PLIEGOMERGE
3533,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,0002. SALUD MATERNO NEONATAL,3033304. ACCESO AL SISTEMA DE REFERENCIA INSTI...,5000052. MEJORAMIENTO DEL ACCESO AL SISTEMA DE...,...,044. SALUD INDIVIDUAL,0096. ATENCION MEDICA BASICA,0033304. ACCESO AL SISTEMA DE REFERENCIA INSTI...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",5. DE VEHICULOS,1. DE VEHICULOS,60000,440
3538,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,1001. PRODUCTOS ESPECIFICOS PARA DESARROLLO IN...,3033254. NIÑOS Y NIÑAS CON VACUNA COMPLETA,5000017. APLICACION DE VACUNAS COMPLETAS,...,044. SALUD INDIVIDUAL,0096. ATENCION MEDICA BASICA,0033254. NIÑOS CON VACUNA COMPLETA,3. BIENES Y SERVICIOS,1. COMPRA DE BIENES,6. REPUESTOS Y ACCESORIOS,1. REPUESTOS Y ACCESORIOS,99. OTROS ACCESORIOS Y REPUESTOS,20000,440
3543,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,1001. PRODUCTOS ESPECIFICOS PARA DESARROLLO IN...,3033254. NIÑOS Y NIÑAS CON VACUNA COMPLETA,5000017. APLICACION DE VACUNAS COMPLETAS,...,044. SALUD INDIVIDUAL,0096. ATENCION MEDICA BASICA,0033254. NIÑOS CON VACUNA COMPLETA,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",5. DE VEHICULOS,1. DE VEHICULOS,20000,440
3544,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,1001. PRODUCTOS ESPECIFICOS PARA DESARROLLO IN...,3033254. NIÑOS Y NIÑAS CON VACUNA COMPLETA,5000017. APLICACION DE VACUNAS COMPLETAS,...,044. SALUD INDIVIDUAL,0096. ATENCION MEDICA BASICA,0033254. NIÑOS CON VACUNA COMPLETA,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",7. DE MAQUINARIAS Y EQUIPOS,1. DE MAQUINARIAS Y EQUIPOS,16000,440
6273,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,9002. ASIGNACIONES PRESUPUESTARIAS QUE NO RESU...,3999999. SIN PRODUCTO,5000936. MANTENIMIENTO DE INFRAESTRUCTURA PUBLICA,...,006. GESTION,0010. INFRAESTRUCTURA Y EQUIPAMIENTO,0332513. MANTENIMIENTO DE LA INFRAESTRUCTURA D...,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...","2. DE EDIFICACIONES, OFICINAS Y ESTRUCTURAS","1. DE EDIFICACIONES, OFICINAS Y ESTRUCTURAS",35458836,441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226601,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,464. GOBIERNO REGIONAL DE LA PROVINCIA CONSTIT...,303. COMITÉ DE ADMINISTRACIÓN DEL FONDO EDUCAT...,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,9002. ASIGNACIONES PRESUPUESTARIAS QUE NO RESU...,3999999. SIN PRODUCTO,5000947. MANTENIMIENTO PREVENTIVO DE INSTITUCI...,...,047. EDUCACION BASICA,0010. INFRAESTRUCTURA Y EQUIPAMIENTO,0006305. INFRAESTRUCTURA EDUCATIVA,3. BIENES Y SERVICIOS,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",7. DE MAQUINARIAS Y EQUIPOS,1. DE MAQUINARIAS Y EQUIPOS,5500

In [10]:
bd_copia = bd_copia1.copy()
bd_copia["PLIEGO1"] =bd_copia["PLIEGO"].str[:3]
bd_copia.rename(columns = {"PLIEGO1":"PLIEGOMERGE"},inplace = True)
bd_copia.drop(columns=['TIPO_GEN_ING'], inplace=True)
bd_copia = bd_copia.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGOMERGE', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA']).agg({"PMG_2025":"sum", "PMG_2026":"sum", "PMG_2027":"sum"}).reset_index() #LAS VARIABLES DE TU BASE
bd_copia.info()
bd_copia.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NIVEL_GOBIERNO  45 non-null     object
 1   SECTOR          45 non-null     object
 2   PLIEGOMERGE     45 non-null     object
 3   FUENTE          45 non-null     object
 4   RUBRO           45 non-null     object
 5   CATEGORIA_GTO   45 non-null     object
 6   GENERICA        45 non-null     object
 7   PMG_2025        45 non-null     int64 
 8   PMG_2026        45 non-null     int64 
 9   PMG_2027        45 non-null     int64 
dtypes: int64(3), object(7)
memory usage: 3.6+ KB


,NIVEL_GOBIERNO,SECTOR,PLIEGOMERGE,FUENTE,RUBRO,CATEGORIA_GTO,GENERICA,PMG_2025,PMG_2026,PMG_2027
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,1620705,1650902,1715265
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,285,285,285
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,43633259,45535980,49705141
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,22814620,21998440,21426080
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,442,5. RECURSOS DETERMINADOS,15. FONDO DE COMPENSACION REGIONAL - FONCOR,5. GASTOS CORRIENTES,3. BIENES Y SERVICIOS,2173179,2219739,2110312


In [11]:
Seg_2024_formato_group = Seg_2024_formato.groupby(by=['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGOMERGE', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA']).agg({"PIA":"sum"}).reset_index() #LAS VARIABLES DE TU BASE
Seg_2024_formato_group = Seg_2024_formato_group.merge(bd_copia, on = ['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGOMERGE',  'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'], how = "left") # MERGE DEL GROUP BY CON LA BASE ESTIMADA

Seg_2024_formato = Seg_2024_formato.merge(Seg_2024_formato_group, on = ['NIVEL_GOBIERNO', 'SECTOR', 'PLIEGOMERGE', 'FUENTE', 'RUBRO', 'CATEGORIA_GTO', 'GENERICA'], how = "left") #MERGE DE LA BASE SEGUIMIENTO CON LA ANTERIOR

Seg_2024_formato["RATIO"] = Seg_2024_formato["PIA_x"]/Seg_2024_formato["PIA_y"] #RATIO

Seg_2024_formato["PMG_2025_E"] = round(Seg_2024_formato["PMG_2025"]*Seg_2024_formato["RATIO"],0)
Seg_2024_formato["PMG_2026_E"] = round(Seg_2024_formato["PMG_2026"]*Seg_2024_formato["RATIO"],0)
Seg_2024_formato["PMG_2027_E"] = round(Seg_2024_formato["PMG_2027"]*Seg_2024_formato["RATIO"],0)
Seg_2024_formato.drop(columns=['PIA_x', 'PIA_y', 'RATIO', 'PLIEGOMERGE'], inplace=True)
Seg_2024_formato.info()
Seg_2024_formato.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NIVEL_GOBIERNO      383 non-null    object 
 1   SECTOR              383 non-null    object 
 2   PLIEGO              383 non-null    object 
 3   EJECUTORA           383 non-null    object 
 4   FUENTE              383 non-null    object 
 5   RUBRO               383 non-null    object 
 6   CATEGORIA_GTO       383 non-null    object 
 7   PROGRAMA_PPTAL      383 non-null    object 
 8   PRODUCTO_PROYECTO   383 non-null    object 
 9   ACT_OBRA_ACCINV     383 non-null    object 
 10  FUNCION             383 non-null    object 
 11  DIVISION_FUNCIONAL  383 non-null    object 
 12  GRUPO_FUNCIONAL     383 non-null    object 
 13  FINALIDAD           383 non-null    object 
 14  GENERICA            383 non-null    object 
 15  SUBGENERICA         383 non-null    object 
 16  SUBGENER

,NIVEL_GOBIERNO,SECTOR,PLIEGO,EJECUTORA,FUENTE,RUBRO,CATEGORIA_GTO,PROGRAMA_PPTAL,PRODUCTO_PROYECTO,ACT_OBRA_ACCINV,...,SUBGENERICA,SUBGENERICA_DET,ESPECIFICA,ESPECIFICA_DET,PMG_2025,PMG_2026,PMG_2027,PMG_2025_E,PMG_2026_E,PMG_2027_E
0,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,0002. SALUD MATERNO NEONATAL,3033304. ACCESO AL SISTEMA DE REFERENCIA INSTI...,5000052. MEJORAMIENTO DEL ACCESO AL SISTEMA DE...,...,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",5. DE VEHICULOS,1. DE VEHICULOS,285.0,285.0,285.0,147.0,147.0,147.0
1,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,1001. PRODUCTOS ESPECIFICOS PARA DESARROLLO IN...,3033254. NIÑOS Y NIÑAS CON VACUNA COMPLETA,5000017. APLICACION DE VACUNAS COMPLETAS,...,1. COMPRA DE BIENES,6. REPUESTOS Y ACCESORIOS,1. REPUESTOS Y ACCESORIOS,99. OTROS ACCESORIOS Y REPUESTOS,285.0,285.0,285.0,49.0,49.0,49.0
2,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,1001. PRODUCTOS ESPECIFICOS PARA DESARROLLO IN...,3033254. NIÑOS Y NIÑAS CON VACUNA COMPLETA,5000017. APLICACION DE VACUNAS COMPLETAS,...,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",5. DE VEHICULOS,1. DE VEHICULOS,285.0,285.0,285.0,49.0,49.0,49.0
3,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,440. GOBIERNO REGIONAL DEL DEPARTAMENTO DE AMA...,401. SALUD BAGUA,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,1001. PRODUCTOS ESPECIFICOS PARA DESARROLLO IN...,3033254. NIÑOS Y NIÑAS CON VACUNA COMPLETA,5000017. APLICACION DE VACUNAS COMPLETAS,...,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...",7. DE MAQUINARIAS Y EQUIPOS,1. DE MAQUINARIAS Y EQUIPOS,285.0,285.0,285.0,39.0,39.0,39.0
4,2. GOBIERNOS REGIONALES,99. GOBIERNOS REGIONALES,441. GOBIERNO REGIONAL DEL DEPARTAMENTO DE ANCASH,001. SEDE ANCASH,5. RECURSOS DETERMINADOS,"18. CANON Y SOBRECANON, REGALIAS, RENTA DE ADU...",5. GASTOS CORRIENTES,9002. ASIGNACIONES PRESUPUESTARIAS QUE NO RESU...,3999999. SIN PRODUCTO,5000936. MANTENIMIENTO DE INFRAESTRUCTURA PUBLICA,...,2. CONTRATACION DE SERVICIOS,"4. SERVICIO DE MANTENIMIENTO, ACONDICIONAMIENT...","2. DE EDIFICACIONES, OFICINAS Y ESTRUCTURAS","1. DE EDIFICACIONES, OFICINAS Y ESTRUCTURAS",22814620.0,21998440.0,21426080.0,22814620.0,21998440.0,21426080.0


In [12]:

outputFile = os.path.join(r'Y:\PMG 2025-2027\Outputs',' Propuesta_RD_GG 2.3 Mantenimiento_PMG2025-2027_Ajustada_'+ fecha_corte + ".xlsx")
with pd.ExcelWriter(outputFile) as ew:
    Seg_2024_formato.to_excel(ew, sheet_name="RD GG2.3", index = False)
#%%
        